# Import modules

In [2]:
!pip3 install pdflatex qiskit ipywidgets quantum-decomp pylatexenc import-ipynb
!wget https://raw.githubusercontent.com/hkhetawat/QArithmetic/master/QArithmetic.py
!wget https://raw.githubusercontent.com/hkhetawat/QArithmetic/master/qft.py

Defaulting to user installation because normal site-packages is not writeable


--2022-09-23 10:22:40--  https://raw.githubusercontent.com/hkhetawat/QArithmetic/master/QArithmetic.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13886 (14K) [text/plain]
Saving to: ‘QArithmetic.py.2’

QArithmetic.py.2    100%[===================>]  13,56K  --.-KB/s    in 0s      

2022-09-23 10:22:41 (36,4 MB/s) - ‘QArithmetic.py.2’ saved [13886/13886]

--2022-09-23 10:22:41--  https://raw.githubusercontent.com/hkhetawat/QArithmetic/master/qft.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1820 (1,8K) [text/plain]
Saving

In [8]:
import import_ipynb, qiskit, functools, QArithmetic
import numpy as np
import matplotlib as plt
from itertools import chain
from qiskit import Aer, IBMQ, QuantumRegister, ClassicalRegister, QuantumCircuit, execute, assemble, transpile
from qiskit.providers.ibmq import least_busy
from qiskit.tools.monitor import job_monitor
from qiskit.providers.aer import noise
from qiskit.circuit import Gate
from qiskit.visualization import plot_histogram

In [14]:
class semigroupMembershipOracle():
    def __init__(self, soughtElement, *generators):
        
        self.soughtElement = soughtElement
        self.generators = generators
    
    def substraction():
        '''operador de diferencia'''
        pass
    def addition():
        '''operador de suma'''
        pass
    def product():
        '''operador de producto'''
        pass
    def linearCombination():
        '''operador de combinación lineal'''
        pass
    

In [19]:
test1 = semigroupMembershipOracle(7, 3, 5)
test1.soughtElement
list(test1.generators)

[3, 5]

In [23]:
class groverAlgorithm():
    
    '''El oráculo como init o como herencia?'''
    
    def Difussor():
        '''Crea un difusor en los hilos implicados'''
        pass
    def Circuit():
        '''Ensambla el circuito con las iteraciones necesarias'''
        pass
    

In [24]:
class semigroupMembership(groverAlgorithm):
    
    def dec2binQR(circ, qReg, number):
        '''Toma un valor decimal y un registro, introduce el equiv. binairo en el registro.'''
        
        binExp = bin(number) #−> 0bxxxxx...
            
        if len(qReg)<len(binExp)-2:
            print("error al convertir en binario")
            return
        i=0
        while binExp[-(i+1)]!='b':
            if binExp[-(i+1)]=='1':
              circ.x(qReg[i])
            i += 1